In [19]:
from google.cloud import storage
import pandas as pd
import numpy as np
import h5py
import io

# GCS config
BUCKET_NAME = "inkling-ssvep-emg"
BASE_PREFIX = "EMG-nature/data"

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)


In [20]:
def list_block_paths():
    # Find all folders that have emg_data.hdf5
    blobs = storage_client.list_blobs(BUCKET_NAME, prefix=BASE_PREFIX)
    block_paths = set()
    for blob in blobs:
        if blob.name.endswith("emg_data.hdf5"):
            block_paths.add(blob.name.rsplit('/', 1)[0])  # drop filename
    return sorted(block_paths)

block_paths = list_block_paths()
print("Found blocks:", len(block_paths))
print(block_paths[:4])


Found blocks: 32
['EMG-nature/data/participant_1/participant1_day1_block1', 'EMG-nature/data/participant_1/participant1_day1_block2', 'EMG-nature/data/participant_1/participant1_day2_block1', 'EMG-nature/data/participant_1/participant1_day2_block2']


In [22]:
def load_block_raw(block_path: str) -> pd.DataFrame:
    # Parse identifiers from path
    parts = block_path.split("/")
    participant = parts[-2]
    blockname = parts[-1]

    # --- Load trials.csv ---
    trials_blob = bucket.blob(f"{block_path}/trials.csv")
    trials_bytes = trials_blob.download_as_bytes()
    trials_df = pd.read_csv(io.BytesIO(trials_bytes))

    # --- Load EMG trials from emg_data.hdf5 ---
    emg_blob = bucket.blob(f"{block_path}/emg_data.hdf5")
    emg_bytes = emg_blob.download_as_bytes()

    emg_list = []
    with h5py.File(io.BytesIO(emg_bytes), "r") as h5:
        keys = sorted(h5.keys(), key=lambda k: int(k))  # '0'..'149'

        for k in keys:
            arr = h5[k][()]         # 2D: (samples,16) or (16,samples)
            if arr.ndim != 2:
                raise ValueError(f"{blockname} dataset {k} has ndim={arr.ndim}")

            # Ensure shape = (16, n_samples)
            if arr.shape[1] == 16:
                arr = arr.T         # (16, n_samples)
            elif arr.shape[0] == 16:
                pass                # already (16, n_samples)
            else:
                raise ValueError(f"{blockname} dataset {k} shape={arr.shape}")

            emg_list.append(arr)

    # Sanity: one EMG array per trial row
    assert len(trials_df) == len(emg_list), f"Trial mismatch in {blockname}"

    # Attach metadata
    trials_df["participant"] = participant
    trials_df["block"] = blockname
    trials_df["emg"] = emg_list                    # list of (16, n_samples)
    trials_df["emg_len"] = [a.shape[1] for a in emg_list]  # keep length info

    return trials_df


In [23]:
all_dfs = []
for path in block_paths:
    print("Loading:", path)
    df_block = load_block_raw(path)
    all_dfs.append(df_block)

emg_df = pd.concat(all_dfs, ignore_index=True)
print("Final DataFrame shape:", emg_df.shape)
print(emg_df.columns)


Loading: EMG-nature/data/participant_1/participant1_day1_block1
Loading: EMG-nature/data/participant_1/participant1_day1_block2
Loading: EMG-nature/data/participant_1/participant1_day2_block1
Loading: EMG-nature/data/participant_1/participant1_day2_block2
Loading: EMG-nature/data/participant_2/participant2_day1_block1
Loading: EMG-nature/data/participant_2/participant2_day1_block2
Loading: EMG-nature/data/participant_2/participant2_day2_block1
Loading: EMG-nature/data/participant_2/participant2_day2_block2
Loading: EMG-nature/data/participant_3/participant3_day1_block1
Loading: EMG-nature/data/participant_3/participant3_day1_block2
Loading: EMG-nature/data/participant_3/participant3_day2_block1
Loading: EMG-nature/data/participant_3/participant3_day2_block2
Loading: EMG-nature/data/participant_4/participant4_day1_block1
Loading: EMG-nature/data/participant_4/participant4_day1_block2
Loading: EMG-nature/data/participant_4/participant4_day2_block1
Loading: EMG-nature/data/participant_4/p

In [24]:
sig = emg_df["emg"].iloc[0]
print(sig.shape, sig.mean(), sig.std())


(16, 9980) -5.476909e-06 2.4578827e-05


In [38]:
emg_df["emg"].iloc[0].shape


(16, 9980)

In [39]:
emg_df.head()

,Unnamed: 0,row_number,target_position,grasp,trial_no,block,participant,emg,emg_len
0,0,0,2,3,0,participant1_day1_block1,participant_1,"[[3.763498e-05, 1.9842508e-05, 9.071698e-06, 1...",9980
1,1,1,2,3,1,participant1_day1_block1,participant_1,"[[1.0537988e-05, 1.153949e-05, 1.18090165e-05,...",10000
2,2,2,2,3,2,participant1_day1_block1,participant_1,"[[1.6977565e-05, 1.9937088e-05, 2.1830994e-05,...",9960
3,3,3,2,3,3,participant1_day1_block1,participant_1,"[[3.6807487e-06, 3.2587977e-06, 2.339907e-06, ...",10020
4,4,4,2,3,4,participant1_day1_block1,participant_1,"[[1.5383765e-05, 1.8471881e-05, 1.6300444e-05,...",10020


In [40]:
emg_df = emg_df.drop(columns=["participant", "block", "Unnamed: 0"])


In [42]:
emg_df

,row_number,target_position,grasp,trial_no,emg,emg_len
0,0,2,3,0,"[[3.763498e-05, 1.9842508e-05, 9.071698e-06, 1...",9980
1,1,2,3,1,"[[1.0537988e-05, 1.153949e-05, 1.18090165e-05,...",10000
2,2,2,3,2,"[[1.6977565e-05, 1.9937088e-05, 2.1830994e-05,...",9960
3,3,2,3,3,"[[3.6807487e-06, 3.2587977e-06, 2.339907e-06, ...",10020
4,4,2,3,4,"[[1.5383765e-05, 1.8471881e-05, 1.6300444e-05,...",10020
...,...,...,...,...,...,...
4795,145,9,2,0,"[[1.59362e-05, 1.7032327e-05, 1.913987e-05, 2....",9980
4796,146,9,2,1,"[[1.3964933e-06, 1.4381011e-06, 5.3344643e-06,...",10000
4797,147,9,2,2,"[[9.107072e-06, 1.3961595e-05, 1.9139401e-05, ...",10020
4798,148,9,2,3,"[[4.061428e-05, 3.564699e-05, 3.1583953e-05, 2...",10020


In [47]:
emg_df = emg_df.rename(columns={
    "row_number": "trial_index",
    "target_position": "arm_position",
    "trial_no": "repeat_within_position"
})

In [58]:
emg_df['grasp'].value_counts()

grasp
3    800
1    800
4    800
5    800
2    800
6    800
Name: count, dtype: int64

In [62]:
emg_df['grasp'].value_counts()

grasp
3    800
1    800
4    800
5    800
2    800
6    800
Name: count, dtype: int64

## Choosing power as 'yes' and pick the best 'no' gesture out of the other 5

In [63]:
# Compute RMS feature per trial - RMS is the standard way to quantify muscle activation

# Compute RMS of each trial: sqrt(mean(signal^2)) per channel
rms = np.array([
    np.sqrt(np.mean(trial**2, axis=1))   # shape: (16,)
    for trial in emg_df['emg']
])

# Add mean RMS over channels as a simple activation measure
emg_df['rms_mean'] = rms.mean(axis=1)

In [65]:
# Compare gestures by mean activation
gesture_energy = (
    emg_df.groupby('grasp')['rms_mean']
          .mean()
          .sort_values(ascending=False)
)

print("Average RMS by gesture:\n", gesture_energy)


Average RMS by gesture:
 grasp
1    0.000026
5    0.000026
4    0.000020
3    0.000020
2    0.000019
6    0.000013
Name: rms_mean, dtype: float32


## Power and open are too similar -> power = Yes, Rest = No

In [66]:
df_binary = emg_df[emg_df['grasp'].isin([1, 6])].copy()

df_binary['label'] = (df_binary['grasp'] == 1).astype(int)

print(df_binary['label'].value_counts())
df_binary.head()


label
1    800
0    800
Name: count, dtype: int64


,trial_index,arm_position,grasp,repeat_within_position,emg,emg_len,rms_mean,label
5,5,4,1,0,"[[-4.095496e-05, -3.720362e-05, -1.5032674e-05...",10020,0.000023,1
6,6,4,1,1,"[[2.5097963e-05, 2.3993141e-05, 2.3825929e-05,...",10020,0.000025,1
7,7,4,1,2,"[[1.19977885e-05, 1.2394645e-05, 1.1332447e-05...",9980,0.000024,1
8,8,4,1,3,"[[1.2465078e-05, 1.1931114e-05, 8.8947645e-06,...",10000,0.000023,1
9,9,4,1,4,"[[4.463319e-06, 1.8881104e-06, 4.6771245e-07, ...",9980,0.000024,1


In [70]:
# Keep only YES = power(1) and NO = rest(6)
emg_df = emg_df[emg_df['grasp'].isin([1, 6])].copy()

# Create binary labels
emg_df['label'] = (emg_df['grasp'] == 1).astype(int)

print("New distribution:\n", emg_df_bin['label'].value_counts())
print("Shape:", emg_df.shape)


New distribution:
 label
1    800
0    800
Name: count, dtype: int64
Shape: (1600, 8)


In [71]:
emg_df['grasp'].value_counts()

grasp
1    800
6    800
Name: count, dtype: int64

## Channel Ranking (Best 4 Electrodes)

In [74]:
# Find shortest trial length
min_len = emg_df['emg_len'].min()
print("Using common length:", min_len)

# Create cropped EMG arrays, all shape (16, min_len)
emg_df['emg_cropped'] = emg_df['emg'].apply(lambda arr: arr[:, :min_len])

# Stack into X for ML: (N, 16, min_len)
import numpy as np
X = np.stack(emg_df['emg_cropped'].values)
y = emg_df['label'].values

print("X shape:", X.shape)


Using common length: 9940
X shape: (1600, 16, 9940)


In [ ]:
# Channel ranking with fixed-length data
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

def eval_channels(X, y, ch_idx):
    """Evaluate given channel indices using 5-fold CV logistic regression."""
    X_sub = X[:, ch_idx, :]                 # (N, n_ch, min_len)
    X_flat = X_sub.reshape(len(X_sub), -1)  # flatten time + channels

    clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=200))
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    scores = []
    for train_idx, test_idx in cv.split(X_flat, y):
        clf.fit(X_flat[train_idx], y[train_idx])
        scores.append(clf.score(X_flat[test_idx], y[test_idx]))
    return np.mean(scores)


In [76]:
# Rank individual channels
channel_scores = []
for ch in range(16):
    acc = eval_channels(X, y, [ch])
    channel_scores.append((ch, acc))

channel_scores_sorted = sorted(channel_scores, key=lambda x: x[1], reverse=True)

print("Per-channel ranking (best → worst):")
for ch, sc in channel_scores_sorted:
    print(f"Channel {ch}: {sc:.3f}")


Per-channel ranking (best → worst):
Channel 3: 0.661
Channel 9: 0.651
Channel 10: 0.646
Channel 12: 0.645
Channel 1: 0.639
Channel 0: 0.634
Channel 2: 0.633
Channel 4: 0.631
Channel 6: 0.630
Channel 7: 0.628
Channel 11: 0.620
Channel 8: 0.619
Channel 15: 0.593
Channel 5: 0.579
Channel 13: 0.571
Channel 14: 0.516


In [77]:
# find best 4 among top candidates
from itertools import combinations

top_candidates = [ch for ch, _ in channel_scores_sorted[:8]]  # top 8 singles

combo_results = []
for combo in combinations(top_candidates, 4):
    acc = eval_channels(X, y, list(combo))
    combo_results.append((combo, acc))

combo_results_sorted = sorted(combo_results, key=lambda x: x[1], reverse=True)

print("\nTop 5 four-channel combinations:")
for combo, acc in combo_results_sorted[:5]:
    print(combo, f"→ acc={acc:.3f}")



Top 5 four-channel combinations:
(3, 10, 12, 1) → acc=0.588
(3, 9, 1, 0) → acc=0.586
(3, 12, 1, 2) → acc=0.586
(3, 10, 12, 4) → acc=0.584
(3, 12, 1, 4) → acc=0.584


In [78]:
best_channels = combo_results_sorted[0][0]
print("Best 4 channels:", best_channels)


Best 4 channels: (3, 10, 12, 1)


## Channels to train on = 3, 10, 12, 1 -> same for live EMG electrodes


In [ ]:
# 1) Define your best 4 channels (keep this order for training + live)
best_channels = [3, 10, 12, 1]

# 2) Create a 4-channel version of each trial
emg_df['emg_4ch'] = emg_df['emg_cropped'].apply(
    lambda arr: arr[best_channels, :]    # shape: (4, min_len)
)

# 3) Build X, y for model training (4 channels now instead of 16)
X_4 = np.stack(emg_df['emg_4ch'].values)   # shape: (N, 4, min_len)
y   = emg_df['label'].values

print("X_4 shape:", X_4.shape)
print("y shape:", y.shape, "labels:", np.bincount(y))


X_4 shape: (1600, 4, 9940)
y shape: (1600,) labels: [800 800]


## Train_test Split

In [86]:
from sklearn.model_selection import train_test_split
import numpy as np

# X_4: (N, 4, min_len)
# y:   (N,)

# Convert to Conv1D format → (N, timesteps, channels)
X_time = np.transpose(X_4, (0, 2, 1))  # (N, min_len, 4)

# Split BEFORE scaling (to prevent leakage)
X_train, X_test, y_train, y_test = train_test_split(
    X_time, y, test_size=0.2, stratify=y, random_state=42
)

print("Train:", X_train.shape, "Test:", X_test.shape)
print("Train labels:", np.bincount(y_train))
print("Test labels:", np.bincount(y_test))


Train: (1280, 9940, 4) Test: (320, 9940, 4)
Train labels: [640 640]
Test labels: [160 160]


## Scaling (fit on train only)

In [87]:
from sklearn.preprocessing import StandardScaler

# Flatten time + channels to apply scaler
nsamp_train, t, ch = X_train.shape
nsamp_test = X_test.shape[0]

X_train_flat = X_train.reshape(nsamp_train, t*ch)
X_test_flat  = X_test.reshape(nsamp_test,  t*ch)

# Standardize: fit only on training data
scaler = StandardScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)
X_test_scaled_flat  = scaler.transform(X_test_flat)


## reshape back to conv1D

In [88]:
# Back into (N, timesteps, channels)
X_train_scaled = X_train_scaled_flat.reshape(nsamp_train, t, ch)
X_test_scaled  = X_test_scaled_flat.reshape(nsamp_test,  t, ch)

print("Scaled shapes → Train:", X_train_scaled.shape,
      " Test:", X_test_scaled.shape)


Scaled shapes → Train: (1280, 9940, 4)  Test: (320, 9940, 4)


## Build the model (1D CNN)

In [89]:
from tensorflow.keras import Sequential, layers

# Conv1D expects: (timesteps, channels)
input_shape = X_train_scaled.shape[1:]  # (min_len, 4)

model = Sequential([
    layers.Input(shape=input_shape),

    # Block 1
    layers.Conv1D(32, kernel_size=7, padding='same', activation='relu'),
    layers.MaxPool1D(pool_size=4),

    # Block 2
    layers.Conv1D(64, kernel_size=5, padding='same', activation='relu'),
    layers.MaxPool1D(pool_size=4),

    # Block 3
    layers.Conv1D(128, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPool1D(pool_size=4),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # binary YES/NO output
])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 9940, 32)       │           928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 2485, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 2485, 64)       │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 621, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 621, 128)       │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 155, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 19840)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │     1,269,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,306,945 (4.99 MB)

 Trainable params: 1,306,945 (4.99 MB)

 Non-trainable params: 0 (0.00 B)

In [92]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', 'recall', 'precision']
)

In [93]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

In [95]:
history = model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],
    verbose=1,
)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.8672 - loss: 0.2785 - precision: 0.7971 - recall: 0.9802 - val_accuracy: 0.9570 - val_loss: 0.1960 - val_precision: 0.9247 - val_recall: 1.0000
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - accuracy: 0.9834 - loss: 0.0687 - precision: 0.9784 - recall: 0.9881 - val_accuracy: 0.9883 - val_loss: 0.0355 - val_precision: 0.9853 - val_recall: 0.9926
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.9932 - loss: 0.0212 - precision: 0.9940 - recall: 0.9921 - val_accuracy: 0.9961 - val_loss: 0.0224 - val_precision: 1.0000 - val_recall: 0.9926
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.9980 - loss: 0.0120 - precision: 0.9980 - recall: 0.9980 - val_accuracy: 0.9922 - val_loss: 0.0219 - val_precision: 0.9926 - val_recall: 0.9926
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.9961 - loss: 0.0128 - precision: 0.9960 - recall: 0.9960 - val_accuracy: 0.9922 - val_loss: 0.

## Evaluate

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score

print("\n===== Test Performance =====")
results = model.evaluate(X_test_scaled, y_test, verbose=0)

for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")

# Predictions
y_prob = model.predict(X_test_scaled).ravel()
y_pred = (y_prob >= 0.5).astype(int)  # threshold = 0.5

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=["rest (0)", "power (1)"]
))

prec = precision_score(y_test, y_pred)
rec  = recall_score(y_test, y_pred)
print(f"\nPrecision (power=1): {prec:.4f}")
print(f"Recall    (power=1): {rec:.4f}")



===== Test Performance =====
loss: 0.0092
compile_metrics: 0.9969
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step

Confusion Matrix:
[[160   0]
 [  1 159]]

Classification Report:
              precision    recall  f1-score   support

    rest (0)       0.99      1.00      1.00       160
   power (1)       1.00      0.99      1.00       160

    accuracy                           1.00       320
   macro avg       1.00      1.00      1.00       320
weighted avg       1.00      1.00      1.00       320


Precision (power=1): 1.0000
Recall    (power=1): 0.9938


## Save model

In [99]:
from pathlib import Path

ARTIFACTS_DIR = Path("artifacts_emg")
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

MODEL_PATH = ARTIFACTS_DIR / "emg_cnn_4ch.h5"
model.save(MODEL_PATH)

print("Model saved:", MODEL_PATH)


Model saved: artifacts_emg/emg_cnn_4ch.h5


## Save scaler

In [100]:
import joblib

SCALER_PATH = ARTIFACTS_DIR / "emg_scaler.pkl"
joblib.dump(scaler, SCALER_PATH)

print("Scaler saved:", SCALER_PATH)


Scaler saved: artifacts_emg/emg_scaler.pkl
